# Transaction IDS

Every transaction is identified by a unique `VirtualTransactionId` (`vxid`), which is comprised of a backend ID (or `backendID`) and a sequentially-assigned number local to each backend, known as `localXID`.

Non-virtual TransactionIds (`xid`), are assigned sequentially to transactions from a global counter used by all databases within the PostgreSQL cluster. This assignment happens when a transaction *first writes* to the database.

In [ ]:
psql -c "SELECT pg_current_xact_id_if_assigned();"

txids can be compared with each other (in a circle):

<img src="./helpers/txid-comparison.png" alt="drawing" width="900"/>